In [1]:
%matplotlib inline  
 
import csv
import numpy as np
import matplotlib.pyplot as plt
import math


# X 408*3
# Y 408*1
# W 3*1
# (Y-X*W) 408*1
def delta_F(Y, X, W):
    #print('Y',Y.shape)
    #print('X',X.shape)
    #print('W',W.shape)
    #print('(Y-X.dot(W))',(Y-X.dot(W)).shape)
    #print('X.T',X.T.shape)
    return -2*X.T.dot((Y-X.dot(W)))
    
def norm2(w1,w2):
    return math.sqrt(np.sum((w1 - w2)*(w1 - w2)))

def GradientDescent(X,Y,W, lr = 0.001, tolerance = math.pow(10,-5)):    

    error = 1
    roundcount = 0
    while error > tolerance:
        Wlast = W.copy()
        W = W - lr * delta_F(Y,X,W)
        #print(W,Wlast)
        error = norm2(W, Wlast)
        sse = norm2( X.dot(W) , Y)*norm2( X.dot(W) , Y)
        #print('round', roundcount,'error', error,'sse', sse)
        roundcount += 1
    #print('round', roundcount,'error', error,'sse', sse)
    return W, sse

maxcolumn = 408
with open('MEAP93.csv', newline='') as csvfile:
    # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
    rows = csv.DictReader(csvfile)

    mathten = []
    lnchprg = []
    log_expend = []
    for row in rows:
        mathten.append(float(row['math10']))
        lnchprg.append(float(row['lnchprg']))
        log_expend.append(math.log(float(row['expend']),10))

#plt.plot(lnchprg,mathten,'o')

x2 = np.array(lnchprg[:maxcolumn])
x1 = np.array(log_expend[:maxcolumn])
x0 = np.array([1]*maxcolumn)
X = np.vstack((x0,x1,x2)).T
Y = np.array(mathten[:maxcolumn])
Y = Y.reshape(maxcolumn,1)
W_init = np.array([1,1,1]).reshape(3,1)
print('X', X.shape)
print('Y', Y.shape)
print('W', W_init.shape)


W_optimal,sse = GradientDescent(X,Y,W_init, lr = math.pow(10,-10))
print('lr = 1e-10',W_optimal,sse)
W_optimal,sse = GradientDescent(X,Y,W_init, lr = math.pow(10,-5))
print('lr = 1e-10',W_optimal,sse)

HW1_bestweight = np.array([-20.36081647, 14.34441037, -0.3045853 ])
HW1_bestweight = HW1_bestweight.reshape(3,1)
HW1_bestweight_sse = norm2( X.dot(HW1_bestweight) , Y)*norm2( X.dot(HW1_bestweight) , Y)
print(HW1_bestweight_sse)

import struct
import numpy as np
import time
from sklearn import svm
from sklearn.neighbors.kde import KernelDensity

def decode_idx3_ubyte(idx3_ubyte_file,dataset_size):
    f = open(idx3_ubyte_file, 'rb').read()
    mem_offset = 16
    images = []
    for i in range(dataset_size):
        if (i+1) % (dataset_size/100) == 0:
            print('#', end='')
        images.append( np.array(struct.unpack_from('>784B', f, mem_offset)).reshape((28, 28)))
        mem_offset += (784)
    return images
    
def decode_idx1_ubyte(idx1_ubyte_file,dataset_size):
    f = open(idx1_ubyte_file, 'rb').read()
    mem_offset = 8
    labels = []
    for i in range(dataset_size):
        if (i+1) % (dataset_size/100) == 0:
            print('#', end='')
        labels.append( struct.unpack_from('>B', f, mem_offset)[0] )
        mem_offset += 1
    return labels


train_image = decode_idx3_ubyte('train-images.idx3-ubyte',60000)
train_label = decode_idx1_ubyte('train-labels.idx1-ubyte',60000)
print('load train done')
test_image = decode_idx3_ubyte('t10k-images.idx3-ubyte',10000)
test_label = decode_idx1_ubyte('t10k-labels.idx1-ubyte',10000)
print('load test done')

from collections import Counter
import time

def KNN(test,train_images,train_labels, K = 1):
    
    result = []
    for i in range(60000):
        sum = np.sum( (train_images[i] - test)*(train_images[i] - test) ) #歐幾里得距離
        result.append((sum,train_labels[i]))
        
    sorted_result = sorted(result,key=lambda t:t[0]) #根據distance 由小到大排序
    
    votes = [x[1] for x in sorted_result[0:K]] #取前K個最近的鄰居的label
    
    vote_counts = Counter(votes)
    top1 = vote_counts.most_common(1) #找出0-9哪個被投票最多次
    
    return top1[0][0]
        
    
def testKNN(K,test_size):
    correct = 0
    wrong = 0
    for i in range(test_size):
        predict_label= KNN(test_image[i], train_image, train_label,K=K)
        #print(predict_label,test_label[i])
        if predict_label == test_label[i]:
            correct += 1.0
        else:
            wrong += 1.0
    print('KNN', K, 'Accuracy', correct/(correct+wrong))

tStart = time.time()
testKNN(1,10000)
tEnd = time.time()
print('KNN1 use {} seconds'.format(tEnd - tStart))
tStart = time.time()
testKNN(5,10000)
tEnd = time.time()
print('KNN5 use {} seconds'.format(tEnd - tStart))


train_num = 20000
test_num = 10000

x_train = [ x.reshape(28*28) for x in train_image]
y_train = train_label
x_test = [ x.reshape(28*28) for x in test_image]
y_test = test_label


tStart = time.time()
# 獲取一個支援向量機模型
print('1')
predictor = svm.SVC(kernel='linear')
# 把資料丟進去
print('2')
predictor.fit(x_train[:train_num], y_train[:train_num])
# 預測結果
print('3')
result = predictor.predict(x_test[:test_num])
# 準確率估計
print('4')
accurancy = np.sum(np.equal(result, y_test[:test_num])) / test_num
print(accurancy)
tEnd = time.time()
print('SVM use {} seconds'.format(tEnd - tStart))


train_num = 60000
test_num = 10000

x_train = [ x.reshape(28*28) for x in train_image]
y_train = train_label
x_test = [ x.reshape(28*28) for x in test_image]
y_test = test_label

class_of_mnist = dict()

#把0-9分成九類
for i, x in enumerate( x_train ):
    if y_train[i] in class_of_mnist:
        class_of_mnist[y_train[i]].append(x)
    else:
        class_of_mnist[y_train[i]] = [ x ]
#列印出每一類有多少個 確定自己的程式對不對
for key in class_of_mnist.keys():
    print(key, len(class_of_mnist[key]))
#算出0-9 的分布機率

tStart = time.time()

kdes = []
for key in class_of_mnist.keys():
    kde = KernelDensity(kernel='gaussian', bandwidth = 0.2).fit(class_of_mnist[key])
    kdes.append( (kde,key) )

correct = 0
wrong = 0
for i, x in enumerate( x_test ):
    result = []
    for kde,key in kdes:
        score = kde.score_samples( [x] )
        result.append( (key, score) )
    #按照分數排序
    sorted_result = sorted(result, key=lambda x: x[1], reverse = True)
    #取最高分統計
    if sorted_result[0][0] == y_test[i]:
        correct += 1
    else:
        wrong += 1
        
print('KDE Accuracy', correct/(correct+wrong))
tEnd = time.time()
print('KDE use {} seconds'.format(tEnd - tStart))



X (408, 3)
Y (408, 1)
W (3, 1)
lr = 1e-10 [[0.99878572]
 [0.99596694]
 [0.67637292]] 114056.76117491025
lr = 1e-10 [[inf]
 [inf]
 [nan]] nan
36753.35513497073


/mnt/nfs/work/yshsu0918/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/mnt/nfs/work/yshsu0918/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in multiply
/mnt/nfs/work/yshsu0918/.local/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in subtract
/mnt/nfs/work/yshsu0918/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


########################################################################################################################################################################################################load train done
########################################################################################################################################################################################################load test done
KNN 1 Accuracy 0.9691
KNN1 use 5508.814007997513 seconds
KNN 5 Accuracy 0.9693
KNN5 use 5512.955764055252 seconds
1
2
3
4
0.9143
SVM use 66.0559253692627 seconds
5 5421
0 5923
4 5842
1 6742
9 5949
2 5958
3 6131
6 5918
7 6265
8 5851
KDE Accuracy 0.3815
KDE use 311.796404838562 seconds
